In [14]:
import pickle
import os

from re import subn
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, average_precision_score
import matplotlib.pyplot as plt

from analyzer import data_cleaner
from analyzer.data_transformation import TfidfDataTransformer, BagOfWordsTransformer, DataTransformer

from imblearn.under_sampling import TomekLinks, RandomUnderSampler, CondensedNearestNeighbour,EditedNearestNeighbours
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, ComplementNB
from sklearn.neural_network import MLPClassifier



In [2]:
# Run this cell if you want to ignore warnings
import warnings
from sklearn.exceptions import ConvergenceWarning
from scipy.linalg import LinAlgWarning
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=LinAlgWarning, module='sklearn')
warnings.simplefilter('ignore', category=ConvergenceWarning)


In [39]:
def load_and_clean_data(filepath, rename_dict=None, vectorizer_output='models/vectorizer.sav', **kwargs) -> (pd.DataFrame, DataTransformer):
    # Load data
    df = pd.read_csv(filepath, **kwargs)

    if rename_dict is not None:
        df.rename(columns=rename_dict, inplace=True)
    #display(df.head(5))
    # Clean data
        # Remove @ mentions
    df['clean_text'] = np.vectorize(data_cleaner.clean_mentions)(df['text'])
        # Remove non alfabet chars
    df['clean_text'] = df['clean_text'].str.replace("[^a-zA-Z#]", " ")
        # Remove short words
    df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    # Transform
    #tr = TfidfDataTransformer()
    tr = BagOfWordsTransformer()
    # Stemming
    df['clean_text'] = tr.stemming(df['clean_text'])

    tr.vectorizer_fit(df['clean_text'])
    df_tfidf = tr.transform(df['clean_text'])
    pickle.dump(tr.vectorizer, open(vectorizer_output, 'wb'))


    return df, df_tfidf, tr

def balance_data(X, y, balancer = RandomUnderSampler(sampling_strategy='not minority',random_state=1337)) -> pd.DataFrame:
    X_balanced, y_balanced = balancer.fit_resample(X, y)
    return X_balanced, y_balanced



def fit_model(X, y, model_type, params, model_output='../models/model.sav', ):
    model = model_type(**params)
    model.fit(X,y)
    if model_output is not None:
        pickle.dump(model, open(model_output, 'wb'))
    return model

proba_models = set(['LogisticRegression', 'BernoulliNB', 'MLPClassifier', 'ComplementNB'])
def predict(model, data):
    model_name = type(model).__name__
    if model_name in proba_models:
        predictions = model.predict_proba(data)
    else:
        predictions = model.predict(data)
    return predictions

def metrics(predictions, true_values, name_prefix="", plot=True):
    predictions_int =  predictions[:,1]>=0.3
    # Calculate metrics
    metrics = {
        'f1_score': f1_score(true_values, predictions_int),
        'confusion_matrix': confusion_matrix(true_values, predictions_int),
        'roc_auc_score': roc_auc_score(true_values, predictions[:,1]),
        'average_precision_score': average_precision_score(true_values, predictions[:,1])
    }
    # Plot?? mby
    if plot:
        ConfusionMatrixDisplay.from_predictions(true_values, predictions_int)
        plt.show()
    # Write to outputfile.
    with open(f'out/results/{name_prefix}_out.out', 'w') as f:
            f.write(str(metrics))
    return metrics

In [16]:
def clean_folders():
    for folder, end in [('models', '.sav'), ('out/results', '.out')]:
        filelist = [ f for f in os.listdir(folder) if f.endswith(end) ]
        for f in filelist:
            os.remove(os.path.join(folder, f))

In [34]:
def analyze_sentence(text, model_file, vectorizer_file):
    text = pd.Series([text])
    #text = np.vectorize(data_cleaner.clean_mentions)(text)
    # Remove non alfabet chars
    text = text.str.replace("[^a-zA-Z#]", " ")
    # Remove short words
    text = text.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    # Transform
    tr = TfidfDataTransformer(vectorizer_file)
    # Stemming
    text = tr.stemming(text)
    df_tfidf = tr.transform(text)
    model = pickle.load(open(f'{model_file}', 'rb'))
    return model.predict_proba(df_tfidf)


array([[0.16896918, 0.83103082]])

In [6]:
def __model_tuning__(X_train, y_train, X_valid, y_valid, model_type,params, output_file, data_label="", verbose=False, plot=False):
    output_name = f'{model_type.__name__}-$-{data_label}-{datetime.now().timestamp()}'
    model_output = f'models/{output_name}.sav'
    outputs = []
    if verbose:
        print(f'Starting tuning of {model_type.__name__} with data labeled: {data_label}')
        print('===============')
    parameter_grid = ParameterGrid(params)
    for i, p in enumerate(parameter_grid):
        param_string = subn("[{}',:]","","".join(str(p).split()))[0]
        if verbose:
            print(f'Params {i+1}/{len(parameter_grid)}: {p}')
        model_output = model_output.replace('$',param_string)
        model = fit_model(X_train, y_train, model_type, params=p, model_output=model_output)
        predictions = predict(model, X_valid)
        output = metrics(predictions, y_valid, output_name.replace('$',param_string),plot=plot)
        outputs.append(output)
        if verbose:
            print(f'Output: {output}')
    if verbose:
        print(f'Tuning ended')
        print('===============')
    return outputs


In [41]:
clean_folders()

In [19]:
verbose = True
random_state = 1337

In [40]:
models = [
    (LogisticRegression, {
        'penalty': ['none', 'l2'],
        'class_weight': [None,'balanced'],
        'n_jobs': [-1],
        'solver': ['newton-cholesky']
    }),
    (BernoulliNB, {
        'alpha' : np.arange(0.0, 1.0, 0.2),
        'fit_prior': [True, False],
        'binarize': [None]

    }),
    (MLPClassifier, {
        'hidden_layer_sizes': [(100,), (100, 100), (50, 50, 50), (50,)],
        'activation': ['identity', 'logistic'],
        'alpha': [0.0001, 0.001]
    }),
    (ComplementNB,{
        'alpha' : np.arange(0.0, 1.0, 0.2),
        'fit_prior': [True, False],
        'norm': [True, False],
    })


]


for model, param_grid in models[:2]:
    if verbose:
        print('=====================')
        print(f'=={model.__name__}==')
        print('=====================')

    df, df_tfidf, tr = load_and_clean_data('data/train.csv', rename_dict={'tweet':'text', 'label':'output'})
    X_train, X_valid, y_train, y_valid = train_test_split(df_tfidf, df['output'],test_size=0.3,random_state=random_state)
    X_train_balanced, y_train_balanced = balance_data(X_train, y_train)

    output = __model_tuning__(X_train_balanced, y_train_balanced, X_valid, y_valid, model, param_grid, 'lin_reg_1' ,'bal', verbose=verbose)
    output2 = __model_tuning__(X_train, y_train, X_valid, y_valid, model, param_grid, 'lin_reg_1' ,'unba', verbose=verbose)


==LogisticRegression==
Starting tuning of LogisticRegression with data labeled: bal
Params 1/4: {'class_weight': None, 'n_jobs': -1, 'penalty': 'none', 'solver': 'newton-cholesky'}
Output: {'f1_score': 0.336046511627907, 'confusion_matrix': array([[6727, 2182],
       [ 102,  578]], dtype=int64), 'roc_auc_score': 0.8648029421668768, 'average_precision_score': 0.3519100884150507}
Params 2/4: {'class_weight': None, 'n_jobs': -1, 'penalty': 'l2', 'solver': 'newton-cholesky'}
Output: {'f1_score': 0.29746401644962306, 'confusion_matrix': array([[5863, 3046],
       [  29,  651]], dtype=int64), 'roc_auc_score': 0.9162075858517165, 'average_precision_score': 0.517053932610335}
Params 3/4: {'class_weight': 'balanced', 'n_jobs': -1, 'penalty': 'none', 'solver': 'newton-cholesky'}
Output: {'f1_score': 0.336046511627907, 'confusion_matrix': array([[6727, 2182],
       [ 102,  578]], dtype=int64), 'roc_auc_score': 0.8648029421668768, 'average_precision_score': 0.3519100884150507}
Params 4/4: {'cla

In [9]:
# Balancer test

In [25]:
model1_grid= {
    'penalty': ['none', 'l2'],
    'class_weight': [None, 'balanced'],
    'n_jobs': [-1],
    'solver': ['newton-cholesky'],
    'max_iter': [500]
}
model1 = LogisticRegression
df, df_tfidf, tr = load_and_clean_data('data/train.csv', rename_dict={'tweet':'text', 'label':'output'})
X_train, X_valid, y_train, y_valid = train_test_split(df_tfidf, df['output'],test_size=0.3,random_state=random_state)

balancers = [TomekLinks(sampling_strategy='majority'), TomekLinks(sampling_strategy='not minority'),  RandomUnderSampler(), SMOTE(sampling_strategy='minority'),CondensedNearestNeighbour()]
results = []
for b in balancers[:3]:
    X_train_balanced, y_train_balanced = balance_data(X_train, y_train, b)
    output = __model_tuning__(X_train_balanced, y_train_balanced, X_valid, y_valid, model1, model1_grid, 'lin_reg_1' ,'bal', verbose=False)

    results.append(f'Balancer {str(b)} : {max(output, key=lambda x: x["roc_auc_score"])}')

print('========')


KeyboardInterrupt: 

In [21]:
for r in results:
    print(r)

Balancer TomekLinks(sampling_strategy='majority') : {'f1_score': 0.389544150462225, 'confusion_matrix': array([[7063, 1846],
       [  69,  611]], dtype=int64), 'roc_auc_score': 0.9374362013297856, 'average_precision_score': 0.6739797660178359}
Balancer TomekLinks(sampling_strategy='not minority') : {'f1_score': 0.389544150462225, 'confusion_matrix': array([[7063, 1846],
       [  69,  611]], dtype=int64), 'roc_auc_score': 0.9374362013297856, 'average_precision_score': 0.6739797660178359}
Balancer RandomUnderSampler() : {'f1_score': 0.24647364513734224, 'confusion_matrix': array([[4865, 4044],
       [  16,  664]], dtype=int64), 'roc_auc_score': 0.9288538358434629, 'average_precision_score': 0.6292684522647592}


In [36]:

test = 'bad bad i hate this ugly thingy libtard'
test2 = 'i like this its beautyful, nice job'
m = 'LogisticRegression-class_weightNonemax_iter500n_jobs-1penaltynonesolvernewton-cholesky-bal-1680028951.834127.sav'
analyze_sentence(test2, model_file=f'models/{m}', vectorizer_file='models/vectorizer.sav')


array([[0.7871613, 0.2128387]])